In [1]:
import datetime
import os
import argparse
import torch

from rlpyt.samplers.collections import TrajInfo
from rlpyt.runners.minibatch_rl import MinibatchRlEval, MinibatchRl
from rlpyt.samplers.serial.sampler import SerialSampler
from rlpyt.utils.logging.context import logger_context

from dreamer_agent import DMCDreamerAgent
from algorithm import Dreamer
from envs.dmc import DeepMindControl
from envs.time_limit import TimeLimit
from envs.action_repeat import ActionRepeat
from envs.normalize_actions import NormalizeActions
from envs.wrapper import make_wapper

2024-01-09 07:20:46.138207: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-09 07:20:46.912839: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def build_and_train(
    log_dir,
    game="cartpole_balance",
    run_ID=1,
    cuda_idx=None,
    eval=False,
    save_model="last",
    load_model_path=None,
):
    params = torch.load(load_model_path) if load_model_path else {}
    agent_state_dict = params.get("agent_state_dict")
    optimizer_state_dict = params.get("optimizer_state_dict")
    action_repeat = 2
    factory_method = make_wapper(
        DeepMindControl,
        [ActionRepeat, NormalizeActions, TimeLimit],
        [dict(amount=action_repeat), dict(), dict(duration=1000 / action_repeat)],
    )
    sampler = SerialSampler(
        EnvCls=factory_method,
        TrajInfoCls=TrajInfo,
        env_kwargs=dict(name=game),
        eval_env_kwargs=dict(name=game),
        batch_T=1,
        batch_B=1,
        max_decorrelation_steps=0,
        eval_n_envs=10,
        eval_max_steps=int(10e3),
        eval_max_trajectories=5,
    )
    algo = Dreamer(initial_optim_state_dict=optimizer_state_dict)  # Run with defaults.
    agent = DMCDreamerAgent(
        train_noise=0.3,
        eval_noise=0,
        expl_type="additive_gaussian",
        expl_min=None,
        expl_decay=None,
        initial_model_state_dict=agent_state_dict,
    )
    runner_cls = MinibatchRlEval if eval else MinibatchRl
    runner = runner_cls(
        algo=algo,
        agent=agent,
        sampler=sampler,
        n_steps=1000000,
        log_interval_steps=1e3,
        affinity=dict(cuda_idx=cuda_idx),
    )
    config = dict(game=game)
    name = "dreamer_" + game
    with logger_context(
        log_dir,
        run_ID,
        name,
        config,
        snapshot_mode=save_model,
        override_prefix=True,
        use_summary_writer=True,
    ):
        runner.train()

In [3]:
log_dir = os.path.abspath('/home/eddy/Projects/RL_project/logs_human')

build_and_train(
        log_dir,
        game="walker_walk ",
        run_ID=3,
        cuda_idx=0,
        eval=False,
        save_model="last",
        load_model_path=False,
    )

2024-01-09 07:20:48.164593  | dreamer_walker_walk_3 Runner  master CPU affinity: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11].
2024-01-09 07:20:48.165370  | dreamer_walker_walk_3 Runner  master Torch threads: 6.


using seed 1490
2024-01-09 07:20:49.661459  | dreamer_walker_walk_3 Sampler decorrelating envs, max steps: 0
2024-01-09 07:20:49.662476  | dreamer_walker_walk_3 Serial Sampler initialized.
2024-01-09 07:20:49.662881  | dreamer_walker_walk_3 Running 1000000 iterations of minibatch RL.
2024-01-09 07:20:51.107337  | dreamer_walker_walk_3 Initialized agent model on device: cuda:0.
DAI CHE SIAMO VICINIIIIIIII
size passato a SequenceNStepReturnBuffer = 300000
2024-01-09 07:20:51.109104  | dreamer_walker_walk_3 Optimizing over 1000 iterations.


/home/eddy/.local/lib/python3.10/site-packages/torch/optim/adam.py:33: UserWarning: optimizer contains a parameter group with duplicate parameters; in future, this will cause an error; see github.com/pytorch/pytorch/issues/40967 for more information
  super().__init__(params, defaults)


2024-01-09 07:21:00.913938  | dreamer_walker_walk_3 itr #999 saving snapshot...
2024-01-09 07:21:00.951941  | dreamer_walker_walk_3 itr #999 saved
2024-01-09 07:21:00.958153  | -----------------------------  ----------
2024-01-09 07:21:00.958742  | Diagnostics/NewCompletedTrajs     2
2024-01-09 07:21:00.959143  | Diagnostics/StepsInTrajWindow  1000
2024-01-09 07:21:00.959632  | Diagnostics/Iteration           999
2024-01-09 07:21:00.960082  | Diagnostics/CumTime (s)           9.8431
2024-01-09 07:21:00.960788  | Diagnostics/CumSteps           1000
2024-01-09 07:21:00.961512  | Diagnostics/CumCompletedTrajs     2
2024-01-09 07:21:00.961976  | Diagnostics/CumUpdates            0
2024-01-09 07:21:00.962464  | Diagnostics/StepsPerSecond      101.594
2024-01-09 07:21:00.962843  | Diagnostics/UpdatesPerSecond      0
2024-01-09 07:21:00.963286  | Diagnostics/ReplayRatio           0
2024-01-09 07:21:00.963628  | Diagnostics/CumReplayRatio        0
2024-01-09 07:21:00.963909  | Length/Average  

Imagination: 100%|██████████| 100/100 [02:03<00:00,  1.23s/it]


...

2024-01-09 18:37:20.193197  | dreamer_walker_walk_3 itr #290999 saving snapshot...
2024-01-09 18:37:20.313468  | dreamer_walker_walk_3 itr #290999 saved
2024-01-09 18:37:20.321517  | -----------------------------  ----------------
2024-01-09 18:37:20.322098  | Diagnostics/NewCompletedTrajs       2
2024-01-09 18:37:20.322435  | Diagnostics/StepsInTrajWindow   50000
2024-01-09 18:37:20.322790  | Diagnostics/Iteration          290999
2024-01-09 18:37:20.323236  | Diagnostics/CumTime (s)         40589.2
2024-01-09 18:37:20.323983  | Diagnostics/CumSteps           291000
2024-01-09 18:37:20.324541  | Diagnostics/CumCompletedTrajs     582
2024-01-09 18:37:20.325170  | Diagnostics/CumUpdates              0
2024-01-09 18:37:20.325751  | Diagnostics/StepsPerSecond          7.08743
2024-01-09 18:37:20.326397  | Diagnostics/UpdatesPerSecond        0
2024-01-09 18:37:20.326722  | Diagnostics/ReplayRatio             0
2024-01-09 18:37:20.327452  | Diagnostics/CumReplayRatio          0
2024-01-09 18

Imagination: 100%|██████████| 100/100 [02:07<00:00,  1.28s/it]


2024-01-09 18:39:38.210622  | dreamer_walker_walk_3 itr #291999 saving snapshot...
2024-01-09 18:39:38.330439  | dreamer_walker_walk_3 itr #291999 saved
2024-01-09 18:39:38.338921  | -----------------------------  ---------------
2024-01-09 18:39:38.339564  | Diagnostics/NewCompletedTrajs       2
2024-01-09 18:39:38.340119  | Diagnostics/StepsInTrajWindow   50000
2024-01-09 18:39:38.340562  | Diagnostics/Iteration          291999
2024-01-09 18:39:38.341316  | Diagnostics/CumTime (s)         40727.2
2024-01-09 18:39:38.341657  | Diagnostics/CumSteps           292000
2024-01-09 18:39:38.342073  | Diagnostics/CumCompletedTrajs     584
2024-01-09 18:39:38.342688  | Diagnostics/CumUpdates              0
2024-01-09 18:39:38.343125  | Diagnostics/StepsPerSecond          7.24548
2024-01-09 18:39:38.343697  | Diagnostics/UpdatesPerSecond        0
2024-01-09 18:39:38.344083  | Diagnostics/ReplayRatio             0
2024-01-09 18:39:38.344623  | Diagnostics/CumReplayRatio          0
2024-01-09 18:

Imagination:  19%|█▉        | 19/100 [00:24<01:43,  1.28s/it]


KeyboardInterrupt: 